In [1]:
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten,Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.applications.efficientnet_v2 import EfficientNetV2L
from tensorflow.keras.applications.efficientnet_v2 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
import numpy as np
import pandas as pd
import os
import cv2
import matplotlib.pyplot as plt
import os

In [2]:
img_size = 128
IMAGE_SIZE = [img_size, img_size]

In [3]:
from google.colab import drive
drive.mount('/thesis')

Mounted at /thesis


In [4]:

train_path='/thesis/MyDrive/thesis/new_clean/train'
test_path= '/thesis/MyDrive/thesis/new_clean/test'
val_path=  '/thesis/MyDrive/thesis/new_clean/test'

In [5]:
x_train=[]
for folder in os.listdir(train_path):
    sub_path=train_path+"/"+folder
    print(sub_path)
    for img in os.listdir(sub_path):
        image_path=sub_path+"/"+img
        img_arr=cv2.imread(image_path)
        img_arr=cv2.resize(img_arr,(img_size,img_size))
        x_train.append(img_arr)
        
x_test=[]
for folder in os.listdir(test_path):
    sub_path=test_path+"/"+folder
    print(sub_path)
    for img in os.listdir(sub_path):
        image_path=sub_path+"/"+img
        img_arr=cv2.imread(image_path)
        img_arr=cv2.resize(img_arr,(img_size,img_size))
        x_test.append(img_arr)

x_val=[]
for folder in os.listdir(val_path):
    sub_path=val_path+"/"+folder
    print(sub_path)
    for img in os.listdir(sub_path):
        image_path=sub_path+"/"+img
        img_arr=cv2.imread(image_path)
        img_arr=cv2.resize(img_arr,(img_size,img_size))
        x_val.append(img_arr)

/thesis/MyDrive/thesis/new_clean/train/Hispa
/thesis/MyDrive/thesis/new_clean/train/Brown_Spot
/thesis/MyDrive/thesis/new_clean/train/Leaf_Blast
/thesis/MyDrive/thesis/new_clean/test/Brown_Spot
/thesis/MyDrive/thesis/new_clean/test/Hispa
/thesis/MyDrive/thesis/new_clean/test/Leaf_Blast
/thesis/MyDrive/thesis/new_clean/test/Brown_Spot
/thesis/MyDrive/thesis/new_clean/test/Hispa
/thesis/MyDrive/thesis/new_clean/test/Leaf_Blast


In [6]:
train_x=np.array(x_train)
test_x=np.array(x_test)
val_x=np.array(x_val)

train_x.shape,test_x.shape,val_x.shape

train_x=train_x/255.0
test_x=test_x/255.0
val_x=val_x/255.0

In [7]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(featurewise_center=False,
    samplewise_center=False,
    featurewise_std_normalization=False,
    samplewise_std_normalization=False,
    zca_whitening=False,
    zca_epsilon=1e-06,
    rotation_range=0,
    width_shift_range=0.0,
    height_shift_range=0.0,
    brightness_range=None,
    shear_range=0.0,
    zoom_range=0.0,
    channel_shift_range=0.0,
    fill_mode="nearest",
    cval=0.0,
    horizontal_flip=False,
    vertical_flip=False,
    rescale=None,
    preprocessing_function=None,
    data_format=None,
    validation_split=0.0,
    dtype=None)
test_datagen = ImageDataGenerator(featurewise_center=False,
    samplewise_center=False,
    featurewise_std_normalization=False,
    samplewise_std_normalization=False,
    zca_whitening=False,
    zca_epsilon=1e-06,
    rotation_range=0,
    width_shift_range=0.0,
    height_shift_range=0.0,
    brightness_range=None,
    shear_range=0.0,
    zoom_range=0.0,
    channel_shift_range=0.0,
    fill_mode="nearest",
    cval=0.0,
    horizontal_flip=False,
    vertical_flip=False,
    rescale=None,
    preprocessing_function=None,
    data_format=None,
    validation_split=0.0,
    dtype=None)
val_datagen = ImageDataGenerator(featurewise_center=False,
    samplewise_center=False,
    featurewise_std_normalization=False,
    samplewise_std_normalization=False,
    zca_whitening=False,
    zca_epsilon=1e-06,
    rotation_range=0,
    width_shift_range=0.0,
    height_shift_range=0.0,
    brightness_range=None,
    shear_range=0.0,
    zoom_range=0.0,
    channel_shift_range=0.0,
    fill_mode="nearest",
    cval=0.0,
    horizontal_flip=False,
    vertical_flip=False,
    rescale=None,
    preprocessing_function=None,
    data_format=None,
    validation_split=0.0,
    dtype=None)


training_set = train_datagen.flow_from_directory(train_path,
                                                 target_size = (img_size, img_size),
                                                 batch_size = 32,
                                                 class_mode = 'sparse')

test_set = test_datagen.flow_from_directory(test_path,
                                            target_size = (img_size, img_size),
                                            batch_size = 32,
                                            class_mode = 'sparse')

val_set = val_datagen.flow_from_directory(val_path,
                                            target_size = (img_size, img_size),
                                            batch_size = 32,
                                            class_mode = 'sparse')

training_set.class_indices
train_y=training_set.classes
test_y=test_set.classes
val_y=val_set.classes
train_y.shape,test_y.shape,val_y.shape

Found 441 images belonging to 3 classes.
Found 76 images belonging to 3 classes.
Found 76 images belonging to 3 classes.


((441,), (76,), (76,))

In [9]:
efficientnet = EfficientNetV2L(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)


473186304/473176280 [==============================] - 2s 0us/step


In [15]:
for layer in efficientnet.layers:
    layer.trainable = False

x = Flatten()(efficientnet.output)

prediction = Dense(3, activation='softmax')(x)


model = Model(inputs=efficientnet.input, outputs=prediction)

# view the structure of the model
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 128, 128, 3  0           []                               
                                )]                                                                
                                                                                                  
 rescaling_1 (Rescaling)        (None, 128, 128, 3)  0           ['input_2[0][0]']                
                                                                                                  
 stem_conv (Conv2D)             (None, 64, 64, 32)   864         ['rescaling_1[0][0]']            
                                                                                                  
 stem_bn (BatchNormalization)   (None, 64, 64, 32)   128         ['stem_conv[0][0]']        

In [16]:
model.compile(
  loss='sparse_categorical_crossentropy',
  optimizer="adam",
  metrics=['accuracy']
)



In [ ]:
for x in range(15):
    print("\n\niteration: "+str(x));
    history = model.fit(
      train_x,
      train_y,
      validation_data=(val_x,val_y),
      epochs=15,  
      batch_size=8,shuffle=True)



iteration: 0
Epoch 1/15
56/56 [==============================] - 5s 84ms/step - loss: 2.0774 - accuracy: 0.4785 - val_loss: 2.7467 - val_accuracy: 0.3289
Epoch 2/15
56/56 [==============================] - 4s 72ms/step - loss: 1.7969 - accuracy: 0.5011 - val_loss: 2.3373 - val_accuracy: 0.2237
Epoch 3/15
56/56 [==============================] - 4s 72ms/step - loss: 1.2545 - accuracy: 0.5261 - val_loss: 1.8521 - val_accuracy: 0.3816
Epoch 4/15
56/56 [==============================] - 4s 72ms/step - loss: 1.1491 - accuracy: 0.5193 - val_loss: 1.6985 - val_accuracy: 0.2368
Epoch 5/15
56/56 [==============================] - 4s 74ms/step - loss: 1.3075 - accuracy: 0.5011 - val_loss: 2.9311 - val_accuracy: 0.3816
Epoch 6/15
56/56 [==============================] - 5s 82ms/step - loss: 1.2731 - accuracy: 0.5147 - val_loss: 2.4375 - val_accuracy: 0.3947
Epoch 7/15
56/56 [==============================] - 5s 81ms/step - loss: 1.6368 - accuracy: 0.5306 - val_loss: 3.2028 - val_accuracy: 0.381